In [ ]:
import xpress as xp
from random import shuffle
import numpy as np
import pandas as pd
import os, logging, sys
from importlib import reload

In [4]:
! echo $XPRESS

In [5]:
def check_model_feasibility(model):
    print('Checking feasibility')
    model.iisfirst(0)
#     model.iisall()
    if model.attributes.numiis == 0:
        pass
    else:
        print(
            "problem is infeasible, {} irreducible infeasible sets (IIS) are found. Please try to relax at least one constraint from each IIS.".format(
                model.attributes.numiis
            )
        )
        for i in range(1, model.attributes.numiis + 1):
            print("IIS {}:".format(i))
            miisrow = []
            miiscol = []
            constrainttype = []
            colbndtype = []
            duals = []
            rdcs = []
            isolationrows = []
            isolationcols = []
            # get data for the first IIS
            model.getiisdata(
                i,
                miisrow,
                miiscol,
                constrainttype,
                colbndtype,
                duals,
                rdcs,
                isolationrows,
                isolationcols,
            )
            for k in miisrow:
                print(k.name)

In [13]:
# # Create logger
# logger = logging.getLogger()
# logger.setLevel(logging.DEBUG)

# # Create STDERR handler
# handler = logging.StreamHandler(sys.stderr)
# # ch.setLevel(logging.DEBUG)

# # Create formatter and add it to the handler
# formatter = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
# handler.setFormatter(formatter)

# # Set STDERR handler as the only handler 
# logger.handlers = [handler]

### Knapsack

In [33]:
m = xp.problem()
m.setlogfile('test.log') # logging

# Params
values = [i for i in range(1,11)]
sizes = [i for i in range(1,11)]
shuffle(sizes)

# Variables
items = [xp.var(name=f"item_{i}", vartype=xp.binary) for i in range(1,11)]
m.addVariable(items)

# Constraints
m.addConstraint( xp.Sum( sizes[i] * items[i] for i in range(10)) <= 10  )

# Objective
m.setObjective(xp.Sum( values[i] * items[i] for i in range(10)), sense=xp.maximize)

In [34]:
# xp.controls.outputlog = 1
# ifPrintSolverLog = 1
# m.setControl('outputlog', ifPrintSolverLog)

In [35]:
m.solve()

In [36]:
print("solution:", m.getSolution())
print(values)
print(sizes)

solution: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[4, 6, 1, 8, 7, 10, 9, 5, 3, 2]


### Freight Transport

In [5]:
m = xp.problem()

### Params ###
n_plants = 5
n_customers = 10
units_per_truck = 100 # must be set such that each plant can send at least n_customers/n_plants trucks
lane_costs = [ [np.random.randint(1,20) for _ in range(n_customers)] for _ in range(n_plants)]
customer_demands = [np.random.randint(50,200) for _ in range(n_customers)]
plant_caps = [np.random.randint(300,500) for _ in range(n_plants)]

assert sum(customer_demands) < sum(plant_caps)

In [6]:
sum(customer_demands), sum(plant_caps)

(1060, 2100)

In [7]:
### Variables ###

# trucks_per_lane
tpl_list=[]
for i in range(n_plants):
    tpl = [xp.var(name=f"plant_{i}_to_cust_{j}", vartype=xp.integer, lb=0, ub=100000) for j in range(n_customers)]
    tpl_list.append( tpl )
    m.addVariable(tpl)
# tpl = [xp.var(name=f"plant_{i}_to_cust_{j}", vartype=xp.integer) for i in range(5) for j in range(10)]

### Constraints ###

# Customer demand met
for j in range(n_customers):
    m.addConstraint( xp.Sum( tpl_list[i][j]*units_per_truck for i in range(n_plants)) >= customer_demands[j] )

# Plants caps adhered to
for i in range(n_plants):
    m.addConstraint( xp.Sum( tpl_list[i][j]*units_per_truck for j in range(n_customers)) <= plant_caps[i] )

# Objective
m.setObjective(xp.Sum( tpl_list[i][j] * lane_costs[i][j] for i in range(n_plants) for j in range(n_customers)), 
               sense=xp.minimize)
m.solve()
print("solution:", m.getSolution())

list(zip([var for sublist in tpl_list for var in sublist], 
         [var for sublist in lane_costs for var in sublist], 
         m.getSolution()))

solution: [-0.0, 2.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, 2.0, -0.0, -0.0, 1.0, -0.0, -0.0, 0.0, -0.0, 2.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 2.0, 1.0, -0.0, -0.0, 0.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, 2.0, -0.0, -0.0, -0.0, -0.0, -0.0]


[(plant_0_to_cust_0, 11, -0.0),
 (plant_0_to_cust_1, 3, 2.0),
 (plant_0_to_cust_2, 10, -0.0),
 (plant_0_to_cust_3, 16, -0.0),
 (plant_0_to_cust_4, 16, -0.0),
 (plant_0_to_cust_5, 14, -0.0),
 (plant_0_to_cust_6, 14, -0.0),
 (plant_0_to_cust_7, 11, -0.0),
 (plant_0_to_cust_8, 8, 1.0),
 (plant_0_to_cust_9, 18, -0.0),
 (plant_1_to_cust_0, 8, 1.0),
 (plant_1_to_cust_1, 16, -0.0),
 (plant_1_to_cust_2, 19, -0.0),
 (plant_1_to_cust_3, 10, -0.0),
 (plant_1_to_cust_4, 11, -0.0),
 (plant_1_to_cust_5, 12, -0.0),
 (plant_1_to_cust_6, 10, 1.0),
 (plant_1_to_cust_7, 6, 2.0),
 (plant_1_to_cust_8, 19, -0.0),
 (plant_1_to_cust_9, 14, -0.0),
 (plant_2_to_cust_0, 4, 1.0),
 (plant_2_to_cust_1, 13, -0.0),
 (plant_2_to_cust_2, 3, -0.0),
 (plant_2_to_cust_3, 3, 0.0),
 (plant_2_to_cust_4, 14, -0.0),
 (plant_2_to_cust_5, 7, 2.0),
 (plant_2_to_cust_6, 17, -0.0),
 (plant_2_to_cust_7, 14, -0.0),
 (plant_2_to_cust_8, 12, -0.0),
 (plant_2_to_cust_9, 6, -0.0),
 (plant_3_to_cust_0, 7, -0.0),
 (plant_3_to_cust_1, 10, -

### Production with Setup Costs

In [11]:
t=1000
avg_vol=10
stock_init=avg_vol
epsilon=1e-5
base_prod_cost=6
setup_cost=3
unit_cost=1
stock_cost=0.1
max_units = [ int(np.random.randint(avg_vol,avg_vol*2)*1.1) for i in range(t) ]
demands = [ np.random.randint(avg_vol, avg_vol*2) for i in range(t) ]
demands[0] = min(demands[0], stock_init+max_units[0])
print(f'Initial Stock: {stock_init}')
print(f'Demands: {demands}')
print(f'Max Units: {max_units}')

m = xp.problem()

Initial Stock: 10
Demands: [16, 11, 18, 19, 13, 16, 10, 15, 19, 14, 13, 11, 10, 11, 15, 17, 10, 11, 17, 13, 19, 18, 16, 15, 12, 16, 11, 18, 17, 13, 14, 16, 19, 15, 12, 15, 16, 19, 10, 10, 15, 13, 15, 12, 17, 18, 18, 15, 18, 14, 14, 18, 19, 11, 10, 16, 17, 17, 13, 19, 12, 10, 11, 15, 14, 19, 12, 13, 12, 18, 10, 14, 12, 11, 16, 18, 16, 19, 12, 18, 16, 15, 14, 18, 18, 10, 15, 19, 14, 11, 14, 11, 12, 11, 13, 17, 12, 10, 19, 18, 10, 19, 19, 10, 19, 14, 12, 10, 13, 15, 10, 16, 19, 16, 19, 10, 18, 19, 17, 13, 15, 13, 11, 12, 15, 12, 16, 16, 15, 10, 12, 13, 10, 13, 19, 18, 16, 18, 16, 14, 18, 19, 17, 18, 16, 10, 15, 14, 13, 11, 17, 14, 10, 18, 13, 17, 13, 17, 17, 19, 10, 15, 15, 19, 19, 11, 18, 12, 11, 17, 16, 18, 18, 19, 14, 16, 18, 17, 15, 10, 11, 15, 16, 17, 11, 10, 18, 10, 18, 16, 15, 13, 17, 14, 18, 18, 18, 10, 16, 15, 11, 11, 19, 11, 18, 14, 15, 10, 18, 10, 17, 15, 17, 18, 11, 16, 16, 10, 18, 13, 12, 11, 10, 15, 17, 13, 18, 17, 11, 17, 16, 10, 11, 10, 15, 14, 19, 16, 18, 11, 19, 19, 19, 

In [12]:
optGap = 0.05
maxRunTimeMin = 20
ifPrintSolverLog = 1

In [13]:
xp.controls.outputlog = 1
ifPrintSolverLog = 1
m.setControl('outputlog', ifPrintSolverLog)

In [14]:
# m.setControl('miprelstop',optGap)
# m.setControl('maxtime',maxRunTimeMin*60)
m.setControl('outputlog', ifPrintSolverLog)

In [15]:
demands[2]=5
demands[4]=max(demands[4], 20)

In [16]:
print(f'Initial Stock: {stock_init}')
print(f'Demands: {demands}')
print(f'Max Units: {max_units}')

Initial Stock: 10
Demands: [16, 11, 5, 19, 20, 16, 10, 15, 19, 14, 13, 11, 10, 11, 15, 17, 10, 11, 17, 13, 19, 18, 16, 15, 12, 16, 11, 18, 17, 13, 14, 16, 19, 15, 12, 15, 16, 19, 10, 10, 15, 13, 15, 12, 17, 18, 18, 15, 18, 14, 14, 18, 19, 11, 10, 16, 17, 17, 13, 19, 12, 10, 11, 15, 14, 19, 12, 13, 12, 18, 10, 14, 12, 11, 16, 18, 16, 19, 12, 18, 16, 15, 14, 18, 18, 10, 15, 19, 14, 11, 14, 11, 12, 11, 13, 17, 12, 10, 19, 18, 10, 19, 19, 10, 19, 14, 12, 10, 13, 15, 10, 16, 19, 16, 19, 10, 18, 19, 17, 13, 15, 13, 11, 12, 15, 12, 16, 16, 15, 10, 12, 13, 10, 13, 19, 18, 16, 18, 16, 14, 18, 19, 17, 18, 16, 10, 15, 14, 13, 11, 17, 14, 10, 18, 13, 17, 13, 17, 17, 19, 10, 15, 15, 19, 19, 11, 18, 12, 11, 17, 16, 18, 18, 19, 14, 16, 18, 17, 15, 10, 11, 15, 16, 17, 11, 10, 18, 10, 18, 16, 15, 13, 17, 14, 18, 18, 18, 10, 16, 15, 11, 11, 19, 11, 18, 14, 15, 10, 18, 10, 17, 15, 17, 18, 11, 16, 16, 10, 18, 13, 12, 11, 10, 15, 17, 13, 18, 17, 11, 17, 16, 10, 11, 10, 15, 14, 19, 16, 18, 11, 19, 19, 19, 1

In [17]:
### Variables ###

# Production
prods=[xp.var(name=f"prod_{i}", vartype=xp.integer, lb=0) for i in range(t)]
m.addVariable(prods)

# Stock
stocks=[xp.var(name=f"stock_{i}", vartype=xp.integer, lb=0) for i in range(t)]
m.addVariable(stocks)

# Curr period prod > 0 indicator
prod_inds = [xp.var(name=f"prod_ind_{i}", vartype=xp.binary) for i in range(t)]
m.addVariable(prod_inds)

# Setup indicator
setup_inds = [xp.var(name=f"setup_ind_{i}", vartype=xp.binary) for i in range(t)]
m.addVariable(setup_inds)

In [18]:
### Constraints ###
for i in range(len(prods)):
    max_prod = prods[i] <= max_units[i]
    m.addConstraint( xp.constraint(max_prod, name = f'prod_{i}_lteq_max_units_{i}_{max_units[i]}') )

# Initial stock
m.addConstraint(xp.constraint(stocks[0] == stock_init+prods[0]-demands[0], 
                              name='init_stock_eq_prod_minus_demand'))

# all other stock constraints
for i in range(1, len(stocks)):
    m.addConstraint(xp.constraint(stocks[i] == stocks[i-1]+prods[i]-demands[i], 
                                      name=f'stock_{i}_eq_prod_minus_demand'))
    m.addConstraint(xp.constraint(stocks[i] >= 0, name=f'stock_{i}_gteq_0'))

# curr prod > 0 
max_prod = max(max_units)
for i in range(t):
    m.addConstraint(xp.constraint(prods[i] <= max_prod*prod_inds[i], 
                                      name=f'prod_ind_{i}_cons'))
    
### Linearized setup indicator constraints ###
# Logic: curr_prod * (1-prev_prod) = setup_indicator
# Linearizing non-linear logic below

# 0th period
m.addConstraint(xp.constraint(prods[0] <= setup_inds[0]*max_prod, 
                                      name=f'setup_ind_0_force_1'))
# 1+
for i in range(1,t):
    m.addConstraint(xp.constraint(prod_inds[i] + (1-prod_inds[i-1]) - 1 <= setup_inds[i], 
                                      name=f'setup_ind_{i}_force_1'))
    m.addConstraint(xp.constraint(prod_inds[i] + (1-prod_inds[i-1]) / 2 >= setup_inds[i], 
                                      name=f'setup_ind_{i}_force_0'))

In [19]:
# Objective
m.setObjective(xp.Sum( stocks[i]*stock_cost + setup_cost*setup_inds[i]
                      + prods[i]*unit_cost + prod_inds[i]*base_prod_cost 
                      for i in range(t)), 
               sense=xp.minimize)
# check_model_feasibility(m)
m.solve()
print("solution:", m.getSolution())

solution: [6.0, 15.0, 20.0, 14.0, 16.0, 11.0, 19.0, 11.0, 13.0, 11.0, 12.0, 11.0, 11.0, 18.0, 13.0, 11.0, 10.0, 15.0, 20.0, 12.0, 19.0, 17.0, 16.0, 15.0, 17.0, 13.0, 17.0, 20.0, 12.0, 13.0, 17.0, 17.0, 13.0, 14.0, 17.0, 18.0, 18.0, 12.0, 15.0, 11.0, 12.0, 11.0, 11.0, 19.0, 16.0, 13.0, 16.0, 17.0, 12.0, 12.0, 14.0, 18.0, 19.0, 18.0, 13.0, 11.000000000000002, 18.0, 13.0, 12.0, 19.0, 13.0, 19.0, 0.0, 19.0, 19.0, 11.0, 11.0, 13.0, 15.0, 15.0, 10.0, 14.0, 12.0, 14.0, 18.0, 19.0, 20.0, 20.0, 19.0, 14.0, 15.0, 18.0, 18.0, 13.0, 11.0, 11.0, 12.0, 15.0, 12.0, 16.0, 12.0, 13.0, 14.0, 16.0, 15.0, 17.0, 12.0, 19.0, 11.0, 16.0, 17.0, 13.0, 18.0, 11.0, 13.0, 14.0, 14.0, 18.0, 12.0, 20.0, 20.0, 18.0, 18.0, 13.0, 20.0, 20.0, 11.0, 12.0, 14.0, 15.0, 13.0, 14.0, 18.0, 15.0, 11.0, 12.0, 16.0, 11.0, 11.0, 14.0, 11.0, 15.0, 20.0, 12.0, 12.0, 17.0, 11.0, 18.0, 12.0, 15.0, 14.0, 14.0, 13.0, 12.0, 15.0, 10.0, 16.0, 14.0, 15.0, 19.0, 15.0, 19.0, 19.0, 14.0, 20.0, 14.0, 14.0, 16.0, 15.0, 11.0, 13.0, 16.0, 14.0,

In [422]:
# Compiling variable results
vars=[]
for p in prods: vars.append(p)
for s in stocks: vars.append(s)
for pi in prod_inds: vars.append(pi)
for si in setup_inds: vars.append(si)
    
assert len(m.getSolution()) == len(vars), f"{len(m.getSolution())} != {len(vars)}"

In [423]:
# Creating df to analyze results
df=pd.DataFrame(list(zip(vars, m.getSolution())))
df.columns=['var','val']
df['var_str']=df['var'].astype(str)

In [424]:
demands, stock_init

([11,
  19,
  5,
  16,
  20,
  10,
  10,
  11,
  14,
  18,
  17,
  19,
  17,
  10,
  11,
  12,
  11,
  18,
  18,
  13,
  16,
  13,
  14,
  14,
  15,
  11,
  18,
  14,
  16,
  17,
  17,
  16,
  19,
  11,
  18,
  11,
  13,
  10,
  12,
  10,
  19,
  11,
  14,
  13,
  11,
  11,
  15,
  11,
  16,
  16,
  13,
  14,
  10,
  18,
  16,
  11,
  15,
  15,
  12,
  10,
  19,
  18,
  13,
  18,
  16,
  15,
  10,
  10,
  17,
  10,
  19,
  14,
  10,
  10,
  13,
  10,
  19,
  18,
  14,
  17,
  12,
  18,
  12,
  12,
  13,
  17,
  18,
  10,
  11,
  12,
  10,
  10,
  18,
  13,
  12,
  10,
  19,
  11,
  14,
  18,
  18,
  12,
  18,
  16,
  13,
  16,
  14,
  12,
  10,
  12,
  16,
  17,
  19,
  18,
  12,
  12,
  19,
  19,
  12,
  13,
  15,
  12,
  15,
  18,
  14,
  13,
  12,
  15,
  17,
  17,
  18,
  19,
  19,
  17,
  18,
  13,
  18,
  15,
  12,
  14,
  18,
  10,
  18,
  19,
  15,
  10,
  10,
  17,
  18,
  15,
  11,
  13,
  19,
  13,
  10,
  19,
  17,
  10,
  15,
  11,
  10,
  11,
  18,
  10,
  11,
  15,
  13,

In [428]:
df.iloc[:,:]

,var,val,var_str
0,prod_0,4.0,prod_0
1,prod_1,16.0,prod_1
2,prod_2,13.0,prod_2
3,prod_3,15.0,prod_3
4,prod_4,16.0,prod_4
5,prod_5,18.0,prod_5
6,prod_6,0.0,prod_6
7,prod_7,15.0,prod_7
8,prod_8,17.0,prod_8
9,prod_9,20.0,prod_9


## Duality

### Using barrier/interior point method to find all binding constraints
### Dual values cannot be interpreted as 'shadow prices'

In [37]:
### Barrier method
m = xp.problem()
m.setControl('dualize', 1)
m.setControl('presolve', 0) # may remove redundant constraints which we want to identify as binding
m.setControl('crossover', 0) # dont want solver to crossover from barrier --> simplex

x = xp.var(vartype = xp.continuous, lb = 0)
y = xp.var(vartype = xp.continuous, lb = 0)
m.addVariable(x, y)

m.setObjective(2*x + y, sense = xp.maximize)
C1 = x <= 5
C2 = y <= 5
C3 = x + y <= 10
#C4 = x + y <= 100

m.addConstraint(C1, C2, C3)

m.solve('b')

print('Objective value:', m.getObjVal())
print('Optimal solution:', m.getSolution())
print('Dual value:', m.getDual())

Objective value: 15.000000098023236
Optimal solution: [4.999999997819862, 4.999999995251091]
Dual value: [1.3586724552228628, 0.35867246915011114, 0.6413275476158368]


In [36]:
### Simplex
m = xp.problem()

x = xp.var(vartype = xp.continuous, lb = 0)
y = xp.var(vartype = xp.continuous, lb = 0)
m.addVariable(x, y)

m.setObjective(2*x + y, sense = xp.maximize)
C1 = x <= 5
C2 = y <= 5
C3 = x + y <= 10
#C4 = x + y <= 100

m.addConstraint(C1, C2, C3)

m.solve()

print('Objective value:', m.getObjVal())
print('Optimal solution:', m.getSolution())
print('Dual value:', m.getDual())

Objective value: 15.0
Optimal solution: [5.0, 5.0]
Dual value: [2.0, 1.0, 0.0]


## MIP - Finding Shadow Prices for Constraints

In [7]:
b1=5
b2=5
b3=10
b4=19
c_x=2
c_z=16
a_z=10

In [8]:
### Solve primal MIP
m = xp.problem()

x = xp.var(vartype = xp.continuous, lb = 0)
y = xp.var(vartype = xp.continuous, lb = 0)
z = xp.var(vartype = xp.integer, lb = 0)

m.addVariable(x, y, z)

m.setObjective(c_x*x + y + c_z*z, sense = xp.maximize)
C1 = x <= b1
C2 = y <= b2
C3 = x + y <= b3
C4 = x + y + a_z*z <= b4

m.addConstraint(C1, C2, C3, C4)

m.solve()

print('Objective value:', m.getObjVal())
print('Optimal solution:', m.getSolution())
z = m.getSolution()[-1]
print(f'Fixing value of z to {z}')

Objective value: 30.0
Optimal solution: [5.0, 4.0, 1.0]
Fixing value of z to 1.0


In [41]:
b1=5
b2=5 # changing to 6 exposes degeneracy/symmetry of duals
b3=10
b4=21
c_x=2
c_z=16
a_z=10

In [42]:
### Fix integer vars and solve dual
m = xp.problem()
m.setControl('dualize', 1)

# Variables
x = xp.var(vartype = xp.continuous, lb = 0)
y = xp.var(vartype = xp.continuous, lb = 0)
# z = xp.var(vartype = xp.integer, lb = 0)
m.addVariable(x, y)

# Constraints
C1 = x <= b1
C2 = y <= b2
C3 = x + y <= b3
C4 = x + y + a_z*z <= b4
m.addConstraint(C1, C2, C3, C4)

# Objective
m.setObjective(c_x*x + y + c_z*z, sense = xp.maximize)
m.solve()
print('Objective value:', m.getObjVal())
print('Optimal solution:', m.getSolution())
print('Dual value:', m.getDual())
duals=m.getDual()

Objective value: 31.0
Optimal solution: [5.0, 5.0]
Dual value: [2.0, 1.0, 0.0, 0.0]


In [43]:
### Barrier method
m = xp.problem()
m.setControl('dualize', 1)
m.setControl('presolve', 0) # may remove redundant constraints which we want to identify as binding
m.setControl('crossover', 0) # dont want solver to crossover from barrier --> simplex

x = xp.var(vartype = xp.continuous, lb = 0)
y = xp.var(vartype = xp.continuous, lb = 0)
# z = xp.var(vartype = xp.integer, lb = 0)

m.addVariable(x, y)

m.setObjective(c_x*x + y + c_z*z, sense = xp.maximize)

C1 = x <= b1
C2 = y <= b2
C3 = x + y <= b3
C4 = x + y + a_z*z <= b4

m.addConstraint(C1, C2, C3, C4)

m.solve('b')

print('Objective value:', m.getObjVal())
print('Optimal solution:', m.getSolution())
print('Dual value:', m.getDual())

Objective value: 31.000000021225233
Optimal solution: [4.999999999906932, 4.999999998195945]
Dual value: [1.226318341785139, 0.22631834256779496, 0.7736816421768951, 1.6153782895039602e-08]


## Reduced Cost

### Reduced costs found after finding dual values --> only relevant for an LP
### Can solve for relaxed LP to inform column addition / fixing decisions

In [19]:
def reduced_costs(cons_mat, coefs, duals):
    print(np.dot(cons_mat.T, duals))
    print(f'Reduced costs: {coefs - np.dot(cons_mat.T, duals)}')

In [22]:
### Parameters
b1=5
b2=5
b3=10
b4=19
c_x=2
c_y=1.5 # also try >= 1.6 (reduced cost for y) to see y become utilized in objective
c_z=16
a_z=10
barrier=False

### Solve LP
m = xp.problem()
m.setControl('dualize', 1)

# Variables
x = xp.var(vartype = xp.continuous, lb = 0)
y = xp.var(vartype = xp.continuous, lb = 0)
z = xp.var(vartype = xp.continuous, lb = 0)
m.addVariable(x, y, z)

# Constraints
C1 = x <= b1
C2 = y <= b2
C3 = x + y <= b3
C4 = x + y + a_z*z <= b4
m.addConstraint(C1, C2, C3, C4)

# Objective
m.setObjective(c_x*x + c_y*y + c_z*z, sense = xp.maximize)
if barrier:
    m.setControl('presolve', 0) # may remove redundant constraints which we want to identify as binding
    m.setControl('crossover', 0) # dont want solver to crossover from barrier --> simplex
    m.solve('b')
else: m.solve()

print('Objective value:', m.getObjVal())
print('Optimal solution:', m.getSolution())
print('Dual value:', m.getDual())
duals=m.getDual()

Objective value: 32.4
Optimal solution: [5.0, 0.0, 1.4]
Dual value: [0.3999999999999999, 0.0, 0.0, 1.6]


In [23]:
cons_mat = np.array([
    [1,0,0],
    [0,1,0],
    [1,1,0],
    [1,1,a_z],
])
coefs = np.array([c_x,c_y,c_z])
reduced_costs(cons_mat, coefs, duals)

Reduced costs: [ 0.  -0.1  0. ]


#### Slightly different problem

In [24]:
### Parameters
b1=5
b2=5
b3=10
b4=19
b5=19
c_x=2
c_y=1.5 # also try >= 1.6 (reduced cost for y) to see y become utilized in objective
c_z=16
c_w=15
a_z=10
a_w=10
barrier=False

### Solve LP
m = xp.problem()
m.setControl('dualize', 1)

# Variables
x = xp.var(vartype = xp.continuous, lb = 0)
y = xp.var(vartype = xp.continuous, lb = 0)
z = xp.var(vartype = xp.continuous, lb = 0)
w = xp.var(vartype = xp.continuous, lb = 0)
m.addVariable(x, y, z, w)

# Constraints
C1 = x <= b1
C2 = y <= b2
C3 = x + y <= b3
C4 = x + y + a_z*z <= b4
C5 = x + y + a_w*w <= b5
m.addConstraint(C1, C2, C3, C4, C5)

# Objective
m.setObjective(c_x*x + c_y*y + c_z*z + c_w*w, sense = xp.maximize)
if barrier:
    m.setControl('presolve', 0) # may remove redundant constraints which we want to identify as binding
    m.setControl('crossover', 0) # dont want solver to crossover from barrier --> simplex
    m.solve('b')
else: m.solve()

print('Objective value:', m.getObjVal())
print('Optimal solution:', m.getSolution())
print('Dual value:', m.getDual())
duals=m.getDual()

Objective value: 58.9
Optimal solution: [0.0, 0.0, 1.9, 1.9]
Dual value: [0.0, 0.0, 0.0, 1.6, 1.5]


In [25]:
cons_mat = np.array([
    [1,0,0,0],
    [0,1,0,0],
    [1,1,0,0],
    [1,1,a_z,0],
    [1,1,0,a_w]
])

coefs = np.array([c_x,c_y,c_z,c_w])
print(coefs)

[ 2.   1.5 16.  15. ]


In [28]:
reduced_costs(cons_mat, coefs, duals)

[ 3.1  3.1 16.  15. ]
Reduced costs: [-1.1 -1.6  0.   0. ]


In [147]:
print(f'Reducing z by 0.1 allows for increasing y by {a_z * 0.1} b/c a_z={a_z} in tight 4th constraint')
print(f'To make reducing z worthwhile, 1 unit inc in y ({c_y}) must increase obj more than 0.1 unit increase in z ({0.1*c_z})')
print(f'Reduced costs for y = min({c_y} - {c_z/a_z}, 0) = {min(c_y - c_z/a_z,0)}')

Reducing z by 0.1 allows for increasing y by 1.0 b/c a_z=10 in tight 4th constraint
To make reducing z worthwhile, 1 unit inc in y (1.5) must increase obj more than 0.1 unit increase in z (1.6)
Reduced costs for y = min(1.5 - 1.6, 0) = -0.10000000000000009


In [148]:
'''
If the optimal value of a variable is positive (not zero), then the reduced cost is always zero. If the optimal value of a variable is zero and the reduced cost corresponding to the variable is also zero, then there is at least one other corner that is also in the optimal solution. The value of this variable will be positive at one of the other optimal corners
'''

'\nIf the optimal value of a variable is positive (not zero), then the reduced cost is always zero. If the optimal value of a variable is zero and the reduced cost corresponding to the variable is also zero, then there is at least one other corner that is also in the optimal solution. The value of this variable will be positive at one of the other optimal corners\n'

### Column Generation

In [21]:
sizes=[3,5,9]
demands=[10,6,5]
board_len=20
barrier=False

In [22]:
### Solve LP
m = xp.problem()
m.setControl('dualize', 1)

### Adding variables
vars_={}
for i,size in enumerate(sizes):
    vars_[i] = xp.var(vartype = xp.continuous, lb = 0)
    m.addVariable(vars_[i])

### Adding constraints - each constraint corresponds to a single configuration in initial step (board_len / size)
for i,demand in enumerate(demands):
    c = vars_[i]*int(board_len/sizes[i]) >= demand
    m.addConstraint(c)

In [23]:
### Objective
m.setObjective( sum(vars_[i] for i in vars_.keys() )  , sense = xp.minimize)
if barrier:
    m.setControl('presolve', 0) # may remove redundant constraints which we want to identify as binding
    m.setControl('crossover', 0) # dont want solver to crossover from barrier --> simplex
    m.solve('b')
else: m.solve()
print('Objective value:', m.getObjVal())
print('Optimal solution:', m.getSolution())
print('Dual value:', m.getDual())
duals=m.getDual()

Objective value: 5.666666666666667
Optimal solution: [1.6666666666666667, 1.5, 2.5]
Dual value: [0.16666666666666666, 0.25, 0.5]


In [24]:
a=0.166666
b=0.25
c=0.5
a*5+b

1.0833300000000001

In [25]:
c+b+a*2

1.083332

### Generating new column

In [26]:
m = xp.problem()

### Adding variables - integer, one per size - indicates number of that size which will be cut from single board
vars_={}
for i,size in enumerate(sizes):
    vars_[i] = xp.var(vartype = xp.integer, lb = 0)
    m.addVariable(vars_[i])

### Adding constraints - each constraint corresponds to only a single configuration in initial step (boar)
c = sum(vars_[i]*sizes[i] for i in vars_.keys()) <= board_len
m.addConstraint(c)

### Objective
m.setObjective( 1 - sum(vars_[i]*duals[i] for i in vars_.keys() )  , sense = xp.minimize)
if barrier:
    m.setControl('presolve', 0) # may remove redundant constraints which we want to identify as binding
    m.setControl('crossover', 0) # dont want solver to crossover from barrier --> simplex
    m.solve('b')
else: m.solve()
print('Objective value:', m.getObjVal())
print('Optimal solution:', m.getSolution())
new_col_cons_coefs = m.getSolution()
new_col_cons_coefs = np.array([int(c) for c in new_col_cons_coefs])

Objective value: -0.08333333333333237
Optimal solution: [5.0, 1.000000000000004, -3.774758283725532e-15]


### Re-Solving LP Relaxation with new column

In [27]:
cons_mat = np.zeros(shape=(3,3))
for i in range(len(sizes)):
    cons_mat[i,i] = int(board_len/sizes[i])
cons_mat = np.hstack( (cons_mat, np.array(new_col_cons_coefs).reshape(-1,1)) )

### Solve LP
m = xp.problem()
m.setControl('dualize', 1)

### Adding variables
vars_ = [xp.var(vartype = xp.continuous, lb = 0) for i in range(len(sizes) + 1)]
m.addVariable(vars_)

### Adding constraints - each constraint corresponds to a single configuration in initial step (board_len / size)
for i,demand in enumerate(demands):
    c = sum(vars_[j]*cons_mat[i,j] for j in range(cons_mat.shape[1])) >= demand
    m.addConstraint(c)

In [28]:
cons_mat

array([[6., 0., 0., 5.],
       [0., 4., 0., 1.],
       [0., 0., 2., 0.]])

#### Note that dual for size=3 drops from 0.1666 to 0.15 b/c there is now no waste associated with 3s
#### We now get 5 3s from 15 feet (15% of a board per 3) compared to 6 3s from 20 feet (16.666% of a board per 3)

In [29]:
### Objective
m.setObjective( sum(vars_[i] for i in range(len(vars_)) )  , sense = xp.minimize)
if barrier:
    m.setControl('presolve', 0) # may remove redundant constraints which we want to identify as binding
    m.setControl('crossover', 0) # dont want solver to crossover from barrier --> simplex
    m.solve('b')
else: m.solve()
print('Objective value:', m.getObjVal())
print('Optimal solution:', m.getSolution())
print('Dual value:', m.getDual())
duals=m.getDual()

Objective value: 5.5
Optimal solution: [0.0, 1.0, 2.5, 2.0]
Dual value: [0.15000000000000002, 0.25, 0.5]


### Gen new col

In [30]:
m = xp.problem()

### Adding variables - integer, one per size - indicates number of that size which will be cut from single board
vars_={}
for i,size in enumerate(sizes):
    vars_[i] = xp.var(vartype = xp.integer, lb = 0)
    m.addVariable(vars_[i])

### Adding constraints
c = sum(vars_[i]*sizes[i] for i in vars_.keys()) <= board_len
m.addConstraint(c)

### Objective
m.setObjective( 1 - sum(vars_[i]*duals[i] for i in vars_.keys() )  , sense = xp.minimize)
if barrier:
    m.setControl('presolve', 0) # may remove redundant constraints which we want to identify as binding
    m.setControl('crossover', 0) # dont want solver to crossover from barrier --> simplex
    m.solve('b')
else: m.solve()
print('Objective value:', m.getObjVal())
print('Optimal solution:', m.getSolution())
new_col_cons_coefs = m.getSolution()
new_col_cons_coefs = np.array([int(c) for c in new_col_cons_coefs])

Objective value: -0.050000000000000044
Optimal solution: [2.0, 1.0, 1.0]


### Re-Solving

In [31]:
cons_mat = np.hstack( (cons_mat, np.array(new_col_cons_coefs).reshape(-1,1)) )

### Solve LP
m = xp.problem()
m.setControl('dualize', 1)

### Adding variables
vars_ = [xp.var(vartype = xp.continuous, lb = 0) for i in range(cons_mat.shape[1])]
m.addVariable(vars_)

### Adding constraints - each constraint corresponds to a single configuration in initial step (board_len / size)
for i,demand in enumerate(demands):
    c = sum(vars_[j]*cons_mat[i,j] for j in range(cons_mat.shape[1])) >= demand
    m.addConstraint(c)

assert cons_mat.shape[1] == 5

In [32]:
cons_mat

array([[6., 0., 0., 5., 2.],
       [0., 4., 0., 1., 1.],
       [0., 0., 2., 0., 1.]])

In [33]:
### Objective
m.setObjective( sum(vars_[i] for i in range(len(vars_)) )  , sense = xp.minimize)
if barrier:
    m.setControl('presolve', 0) # may remove redundant constraints which we want to identify as binding
    m.setControl('crossover', 0) # dont want solver to crossover from barrier --> simplex
    m.solve('b')
else: m.solve()
print('Objective value:', m.getObjVal())
print('Optimal solution:', m.getSolution())
print('Dual value:', m.getDual())
duals=m.getDual()

Objective value: 5.25
Optimal solution: [0.0, 0.25, 0.0, 0.0, 5.0]
Dual value: [0.15000000000000002, 0.25, 0.44999999999999996]


### Trying to generate new column - none with negative reduced cost so solution is optimal

In [34]:
m = xp.problem()

### Adding variables - integer, one per size - indicates number of that size which will be cut from single board
vars_={}
for i,size in enumerate(sizes):
    vars_[i] = xp.var(vartype = xp.integer, lb = 0)
    m.addVariable(vars_[i])

### Adding constraints
c = sum(vars_[i]*sizes[i] for i in vars_.keys()) <= board_len
m.addConstraint(c)

### Objective
m.setObjective( 1 - sum(vars_[i]*duals[i] for i in vars_.keys() )  , sense = xp.minimize)
if barrier:
    m.setControl('presolve', 0) # may remove redundant constraints which we want to identify as binding
    m.setControl('crossover', 0) # dont want solver to crossover from barrier --> simplex
    m.solve('b')
else: m.solve()
print('Objective value:', m.getObjVal())
print('Optimal solution:', m.getSolution())
new_col_cons_coefs = m.getSolution()
new_col_cons_coefs = np.array([int(c) for c in new_col_cons_coefs])

Objective value: 0.0
Optimal solution: [5.0, 1.0, -0.0]
